In [1]:
import shap
import joblib
import sys,os,time,platform

if platform.node() == 'aizat-fyp':
    sys.path.append(r'/home/aizat/ember')
    sys.path.append(r'/home/aizat/OneDrive/Master Project/Workspace/boosting-ember')
    datasetpath = '/home/aizat/OneDrive/Master Project/Workspace/dataset/ember2018'
else:
    sys.path.append(r'/root/ember')
    sys.path.append(r'/root/boosting-ember')
    datasetpath = '/root/dataset/ember2018/'
import mlflow    
import ember
import pandas as pd
import numpy as np
import lightgbm as lgb
import boostember
from boostember.features_extended import *

boostember.mlflowsetup('https://raw.githubusercontent.com/aizatrosli/boosting-ember/main/mlflow?token=ACRLNIPCA745DENP3KF3XEDAQ7TKE')


True

In [2]:
X_train, y_train, X_test, y_test = ember.read_vectorized_features(datasetpath)
delunlabel = (y_train != -1)

X_train = X_train[delunlabel]
#X_train = pd.DataFrame(X_train, columns=emberfeaturesheader())
y_train = y_train[delunlabel]
X_train.shape, y_train.shape

((600000, 2381), (600000,))

In [3]:
mlflow.set_tracking_uri("https://atlascompanion.live/")
#mlflow.create_experiment("", artifact_location="gs://mlflow_aizat")
mlflow.set_experiment("Demo")

In [4]:
def fetch_logged_data(run_id):
    client = mlflow.tracking.MlflowClient()
    data = client.get_run(run_id).data
    tags = {k: v for k, v in data.tags.items() if not k.startswith("mlflow.")}
    artifacts = [f.path for f in client.list_artifacts(run_id, "model")]
    return data.params, data.metrics, tags, artifacts

model = joblib.load('./model/defaultparams_11_4.pkl')

In [5]:
mlflow.lightgbm.autolog()
with mlflow.start_run(run_name="demo_ember_lightgbm") as run:
    mlflow.set_tags({"description": "Demo",
        "model": "lightgbm",
        "summary": "pretrained params"})
    lgbm_dataset = lgb.Dataset(X_train, y_train)
    lgbm_model = lgb.train(model.params, lgbm_dataset)
    params, metrics, tags, artifacts = fetch_logged_data(run.info.run_id)

2021/04/20 20:50:24 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of lightgbm. If you encounter errors during autologging, try upgrading / downgrading lightgbm to a supported version, or try upgrading MLflow.
Found `num_iterations` in params. Will use it instead of argument
Found `early_stopping_round` in params. Will use it instead of argument


[LightGBM] [Info] Number of positive: 300000, number of negative: 300000
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 3.633904 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 212057
[LightGBM] [Info] Number of data points in the train set: 600000, number of used features: 2333
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


In [8]:
def yield_artifacts(run_id, path=None):
    """Yield all artifacts in the specified run"""
    client = mlflow.tracking.MlflowClient()
    for item in client.list_artifacts(run_id, path):
        if item.is_dir:
            yield from yield_artifacts(run_id, item.path)
        else:
            yield item.path


def fetch_logged_data(run_id):
    """Fetch params, metrics, tags, and artifacts in the specified run"""
    client = mlflow.tracking.MlflowClient()
    data = client.get_run(run_id).data
    # Exclude system tags: https://www.mlflow.org/docs/latest/tracking.html#system-tags
    tags = {k: v for k, v in data.tags.items() if not k.startswith("mlflow.")}
    artifacts = list(yield_artifacts(run_id))
    return {
        "params": data.params,
        "metrics": data.metrics,
        "tags": tags,
        "artifacts": artifacts,
    }

In [11]:
from sklearn.metrics import (roc_auc_score, make_scorer)
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit

params = {
    'boosting_type': ['goss','gbdt','dart'],
    'objective': ['binary'],
    'num_iterations': [1000],
    'learning_rate': [0.05],
    'num_leaves': [512, 1024, 2048, 4096],
    'feature_fraction': [0.5],
    'bagging_fraction': [0.5]
}

mlflow.sklearn.autolog()
mlflow.lightgbm.autolog()

score = make_scorer(roc_auc_score, max_fpr=5e-3)
model = lgb.LGBMClassifier(boosting_type="gbdt", n_jobs=-1, silent=True)
progressive_cv = TimeSeriesSplit(n_splits=3).split(X_train)
grid = GridSearchCV(estimator=model, cv=progressive_cv, param_grid=params, scoring=score, n_jobs=1, verbose=3)




with mlflow.start_run() as run:
    mlflow.set_tags({"description": "Demo",
        "model": "lightgbm",
        "summary": "gridsearch test"})
    grid.fit(X_train, y_train)
    # show data logged in the parent run
    print("========== parent run ==========")
    for key, data in fetch_logged_data(run.info.run_id).items():
        print("\n---------- logged {} ----------".format(key))
        print(data)
        # show data logged in the child runs
        filter_child_runs = "tags.mlflow.parentRunId = '{}'".format(run.info.run_id)
        runs = mlflow.search_runs(filter_string=filter_child_runs)
        param_cols = ["params.{}".format(p) for p in params.keys()]
        metric_cols = ["metrics.mean_test_score"]

        print("\n========== child runs ==========\n")
        pd.set_option("display.max_columns", None)  # prevent truncating columns
        print(runs[["run_id", *param_cols, *metric_cols]])

2021/04/21 06:34:23 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of lightgbm. If you encounter errors during autologging, try upgrading / downgrading lightgbm to a supported version, or try upgrading MLflow.


Fitting 3 folds for each of 12 candidates, totalling 36 fits


Found `num_iterations` in params. Will use it instead of argument


[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5
[CV 1/3] END bagging_fraction=0.5, boosting_type=goss, feature_fraction=0.5, learning_rate=0.05, num_iterations=1000, num_leaves=512, objective=binary; total time= 4.0min


Found `num_iterations` in params. Will use it instead of argument


...

[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5
[CV 2/3] END bagging_fraction=0.5, boosting_type=dart, feature_fraction=0.5, learning_rate=0.05, num_iterations=1000, num_leaves=4096, objective=binary; total time=48.9min


Found `num_iterations` in params. Will use it instead of argument


[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5
[CV 3/3] END bagging_fraction=0.5, boosting_type=dart, feature_fraction=0.5, learning_rate=0.05, num_iterations=1000, num_leaves=4096, objective=binary; total time=57.6min
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5


Found `num_iterations` in params. Will use it instead of argument
2021/04/21 16:34:16 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/root/anaconda3/envs/fyp/lib/python3.8/site-packages/mlflow/sklearn/__init__.py:866: UserWarning: Logging to MLflow failed: cannot pickle 'generator' object"


========== parent run ==========

---------- logged params ----------
{'cv': '<generator object TimeSeriesSplit.split at 0x7f0571860ba0>', 'error_score': 'nan', 'estimator': 'LGBMClassifier()', 'n_jobs': '1', 'param_grid': "{'boosting_type': ['goss', 'gbdt', 'dart'], 'objective': ['binary'], 'num_iterations': [1000], 'learning_rate': [0.05], 'num_leaves': [512, 1024, 2048, 4096], 'feature_fraction': [0.5], 'bagging_fraction': [0.5]}", 'pre_dispatch': '2*n_jobs', 'refit': 'True', 'return_train_score': 'False', 'scoring': 'make_scorer(roc_auc_score, max_fpr=0.005)', 'verbose': '3', 'best_bagging_fraction': '0.5', 'best_boosting_type': 'gbdt', 'best_feature_fraction': '0.5', 'best_learning_rate': '0.05', 'best_num_iterations': '1000', 'best_num_leaves': '4096', 'best_objective': 'binary'}

========== child runs ==========

                              run_id params.boosting_type params.objective  \
0   36db7b14af6d4c08a0e0aa25ec0972b1                 goss           binary   
1   5624fe6d